# CSE688: Practical and Application of Machine Learning - Spring 2021
## Assignment 3-2
### Authors

- B073040018 朱劭璿
- B072010029 陳居廷

### References
https://www.tensorflow.org/tutorials/text/classify_text_with_bert

#### Sentiment analysis on the sentences in the attachment `hw3-2-dataset`. Classify the sentences into two categories: possitive (1) amd negative (0)

In [1]:
import tensorflow as tf
import numpy as np

DATA_DIR = '../hw3-2-dataset/'
train_label = []
train_data = []
with open(DATA_DIR+'training.txt', 'r') as f:
    lines = f.readlines()
for l in lines:
    l = l[:-1].split('\t')
    train_label.append(int(l[0]))
    train_data.append(l[1])
    
train_label = np.array(train_label)
train_data = np.array(train_data)

ds_train = tf.data.Dataset.from_tensor_slices((train_label, train_data))
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(train_label.shape[0])
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

print('Label:', train_label[0])
print('Sentence:', train_data[0])

Label: 1
Sentence: The Da Vinci Code book is just awesome.


In [2]:
import tensorflow_hub as hub
import tensorflow_text as text

tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_type_ids', 'input_word_ids', 'input_mask']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [3]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Pooled Outputs Shape:(1, 768)
Pooled Outputs Values:[-0.92169887 -0.3935343  -0.53931624  0.6825621   0.43848413 -0.14021143
  0.8774711   0.26043335 -0.63112897 -0.9999657  -0.26319948  0.8510528 ]
Sequence Outputs Shape:(1, 128, 768)
Sequence Outputs Values:[[ 0.19451573  0.2514167   0.19075021 ... -0.2484506   0.38568532
   0.13291022]
 [-0.5947868  -0.3942035   0.25245634 ... -0.7694673   1.1564162
   0.32475743]
 [ 0.00641492 -0.15766467  0.5461023  ... -0.17451009  0.60289615
   0.4267223 ]
 ...
 [ 0.21948345 -0.2092707   0.5386833  ...  0.24693541  0.18250957
  -0.44427103]
 [ 0.01080247 -0.4455314   0.35990998 ...  0.31722802  0.23562792
  -0.6307057 ]
 [ 0.2932117  -0.10581884  0.6114755  ...  0.20745835  0.14494641
  -0.35353386]]
